In [24]:
import psycopg2
import requests
import json
import pandas as pd
%run 'D:\\dev\\prod\\metrikaHelper.py'
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p.status, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s AND p.status<230 ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[3], 'metrikapassword': row[4]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}
def getGoalsByCounter(counterId,token):
    return requests.get('https://api-metrika.yandex.ru/management/v1/counter/{0}/goals?oauth_token={1}'.format(counterId,token)).text.encode('utf-8')

def processNumberGoal(goal):
    return [{'ID': goal['id'], u'Ретаргетинговая':goal['is_retargeting'], u'Имя':goal['name'], u'Тип':u'Глубина просмотра', u'Глубина':goal['depth'], u'Домен':goal['domain']}]
def processUrlGoal(goal):
    conditions = []
    for condition in goal['conditions']:
        cond = {}
        if condition['type'] == 'exact':
            cond[u'Тип соответствия'] = u'Точно соответствует'
        elif condition['type'] == 'contain':
            cond[u'Тип соответствия'] = u'Содержит'
        elif condition['type'] == 'start':
            cond[u'Тип соответствия'] = u'Начинается с'
        elif condition['type'] == 'regexp':
            cond[u'Тип соответствия'] = u'Регулярное выражение'
        else:
            cond[u'Тип соответствия'] = u'Другое'
        cond[u'Условие'] = condition['url']
        conditions.append(cond)
    for condition in conditions:
        condition['ID'] = goal['id']
        condition[u'Ретаргетинговая'] = goal['is_retargeting']
        condition[u'Имя'] = goal['name']
        condition[u'Тип'] = u'Просмотр URL' 
        condition[u'Домен'] = goal['domain']
    return conditions
def processActionGoal(goal):
    conditions = []
    for condition in goal['conditions']:
        cond = {}
        if condition['type'] == 'exact':
            cond[u'Тип соответствия'] = u'Точно соответствует'
        elif condition['type'] == 'contain':
            cond[u'Тип соответствия'] = u'Содержит'
        elif condition['type'] == 'start':
            cond[u'Тип соответствия'] = u'Начинается с'
        elif condition['type'] == 'regexp':
            cond[u'Тип соответствия'] = u'Регулярное выражение'
        else:
            cond[u'Тип соответствия'] = u'Другое'
        cond[u'Условие'] = condition['url']
        conditions.append(cond)
    for condition in conditions:
        condition['ID'] = goal['id']
        condition[u'Ретаргетинговая'] = goal['is_retargeting']
        condition[u'Имя'] = goal['name']
        condition[u'Тип'] = u'Событие' 
        condition[u'Предыдущая цель'] = goal['prev_goal_id']
        condition[u'Домен'] = goal['domain']
    return conditions
def processStepGoal(goal):
    d = {}
    d[u'Имя'] = goal['name']
    d['ID'] = goal['id']
    d[u'Ретаргетинговая'] = goal['is_retargeting']
    d[u'Тип'] = u'Составная'
    d[u'Условие'] = goal['steps']
    d[u'Домен'] = goal['domain']
    return d

In [2]:
goalTypes = ['url','step','number','action']

In [25]:
domains = ["russj.ru","organickids.ru","pinakotekaresto.com","prosto-gruz.ru","zbbb.ru","alcotek.ru","edemdom.ru","labotini.ru","realstroy.com","tencon.ru","жк-московский.com",
"народная-23.рф","doorlock.ru","stm66.ru","academoffice.ru","adel-omsk.ru","baxburo.ru","fabslog.ru","ferkon.ru","fosterland.ru","gemainvest.ru","hotel.sipk.ru",
"kerasol.ru","medstrahovka.ru","m-y.ru","nasoscentr.ru","optikdeal.ru","pickup-center.ru","proudmom.ru","ribambelle.ru","rusmaneken.ru","simple-pro.com","teploppu.ru",
"altpremium.ru","mosvipremont.ru","multifin.ru","megakran.ru","energotehmontag.ru","nanosegway.ru","mebmetall.ru","oooptr77.ru","saabparts.su","lodki-piter.ru",
"accessoriesforhome.ru","darion-svet.ru","vtb24leasing.ru","sevastopolstroy.ru"]

In [26]:
allGoals = []
for domain in domains:
    VipProjectInfo = getVipProjectByDomain(domain)[0]
    metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo['metrikalogin'],'yandexPassword':VipProjectInfo['metrikapassword'],'domain':VipProjectInfo['domain'].decode("utf-8")})
    try:
        GoalsInfo = getGoalsByCounter(metrikaCredentials['counterID'][0],metrikaCredentials['Token'])
        goals = json.loads(GoalsInfo)['goals']
        for goal in goals:
            goal['domain'] = domain
        allGoals += goals
    except:
        print domain
df = pd.DataFrame(allGoals)

Invalid login or password
ribambelle.ru


In [27]:
goalsData = []
for goal in allGoals:
    if goal['type'] == 'url':
        goalsData += processUrlGoal(goal)
    elif goal['type'] == 'number':
        goalsData += processNumberGoal(goal)
    elif goal['type'] == 'action':
        goalsData += processActionGoal(goal)

In [17]:
len(goalsData)

446

In [33]:
pd.DataFrame(goalsData).to_clipboard()

UnicodeEncodeError: 'ascii' codec can't encode characters in position 15-21: ordinal not in range(128)